# Introduction

Since micropython is a complete python interpreter, it is really easy to add new functionality to the hardware. However, there are cases, when defining the new functionality in python is not going to make the cut. First, the microcontroller might not have enough free RAM to compile the code, and second, the interpreter needs time for the compilation step, therefore, the execution of the instructions is slowed down. One can, in principle, gain on speed by implementing time critical code in assembly, and then invoking the `@viper` function decorator, but that has its own limitations. E.g., the number of positional arguments is maximised, and no keyword arguments are allowed, just to name a few problems.

I have been 

In what follows, I would like to show that adding your own C module is not difficult at all, especially that the micropython code base is organised in a rather logical fashion. 

I will try to discuss all aspects of micropython in an approachable way. At the end of each chapter, I will present the discussed code in its entirety. You can simply 

I start out with a very simple module and slowly build upon it. At the end of the discussion, we will create a general-purpose math library, similar to numpy.

Obviously, numpy is a huge library, and we are not going to implement all aspects of it. But we will be able to define arrays on which we can do vectorised computations, work with matrices, invert and contract them, fit polynomials to measurement data, and get the Fourier transform of an arbitrary sequence. I hope you will enjoy the ride!

One last comment: I believe, all examples in this document could be implemented with little effort in python itself, and I am definitely not advocating the inclusion of such trivial cases in the firmware. I chose these examples on two grounds: First, they are all simple, almost primitive, but for this very reason, they demonstrate an idea without distraction. Second, having a piece of parallel python code is useful insofar as it tells us what to expect, and we can compare the results of the C implemetation to that of the native firmware.

# Setting up the environment, compiling the code


Beginning with the 1.10 version of micropython, it became quite simple to add a user-defined C module to the firmware. 


-------------------------------

The micropython codebase itself is set up a rather modular way. If you look at the top level directories,

In [5]:
!ls ../../micropython/

ACKNOWLEDGEMENTS    docs      extmod   logo	  py	     tools
CODECONVENTIONS.md  drivers   lib      mpy-cross  README.md
CONTRIBUTING.md     examples  LICENSE  ports	  tests


at least two are of particular interest. Namely, `/py/`, where the python interpreter is implemented, and `/ports/`, which contains the hardware-specific files. 

# Developing your own module

We begin with adding a simple module to the python interpreter. The module will have a single function that takes two numbers, and adds them. As a module, this is not particularly useful, but we can 
. However, I believe, starting with and understanding the simple is a really good way of learning. After getting to know the foundations, it will be rather trivial to extend the module with more useful functionality. 


First I show the file in its entirety, and then discuss the parts.

In [1]:
!less ../simplefunction/simplefunction.c

// Include required definitions first.
#include "py/obj.h"
#include "py/runtime.h"
#include "py/builtin.h"

// This is the function which will be called from Python as example.add_ints(a, b).
STATIC mp_obj_t simplefunction_add_ints(mp_obj_t a_obj, mp_obj_t b_obj) {
    // Extract the ints from the micropython input objects
    int a = mp_obj_get_int(a_obj);
    int b = mp_obj_get_int(b_obj);

    // Calculate the addition and convert to MicroPython object.
    return mp_obj_new_int(a + b);
}
// Define a Python reference to the function above
STATIC MP_DEFINE_CONST_FUN_OBJ_2(simplefunction_add_ints_obj, simplefunction_add_ints);

// Define all properties of the example module.
// Table entries are key/value pairs of the attribute name (a string)
// and the MicroPython object reference.
// All identifiers and strings are written as MP_QSTR_xxx and will be
:simplefunction/simplefunction.c

### Defining user functions

First, we define the function that is going to do the heavy lifting. By passing variables of `mp_obj_t` type, we make sure that the function will be able to accept values from the python console. If you happen to have an internal helper function in your module that is not exposed in python, you can pass whatever type you need. 
Similarly, by returning an object of `mp_obj_t` type, we make the results visible to the interpreter. 

The downside of passing `mp_obj_t`s around is that you cannot simply assign them to usual C variables. This is why we have to invoke the `mp_obj_get_int()` function, and conversely, before returning the results, we have to do a type conversion by calling `mp_obj_new_int()`.

### Referring to user functions

Once we implemented the functionality we need, we have to turn our function into a function object that the python interpreter can work with. This is what happens in the line

```c
STATIC MP_DEFINE_CONST_FUN_OBJ_2(simplefunction_add_ints_obj, simplefunction_add_ints);
```

The first argument is the name of the function object to which our actual function, the last argument, will be bound. Now, these `MP_DEFINE_CONST_FUN_OBJ_*` macros, defined in the header file `py/obj.h`, come in seven flavours, depending on what kind of, and how many arguments the function is supposed to take. In the example above, our function is meant to take two arguments, hence the 2 at the end of the macro name. Functions with 0 to 4 arguments can be bound in this way.

But what, if you want a function with more than four arguments, as is the case many a time in python? In this case, one can make use of the 

```c
MP_DEFINE_CONST_FUN_OBJ_VAR(obj_name, n_args_min, fun_name)
```

macro, where the second argument, an integer, gives the minimum number of arguments. The number of arguments can be bound from above by wrapping the function with 

```c
MP_DEFINE_CONST_FUN_OBJ_VAR_BETWEEN(obj_name, n_args_min, n_args_max, fun_name) 
```

Later we will see, how we can define functions that can also take keyword arguments.

At this point, we are more or less done with the C implementation of our function, but we still have to expose it. This we do by adding a table, an array of key/value pairs to the globals of our module, and bind the table to the `_module_globals` variable by applying the `mP_DEFINE_CONST_DICT` macro. This table should have at least one entry, the name of the module. 

The keys are of the table are turned


```c
STATIC const mp_rom_map_elem_t simplefunction_module_globals_table[] = {
    { MP_ROM_QSTR(MP_QSTR___name__), MP_ROM_QSTR(MP_QSTR_simplefunction) },
    { MP_ROM_QSTR(MP_QSTR_add_ints), MP_ROM_PTR(&simplefunction_add_ints_obj) },
};
STATIC MP_DEFINE_CONST_DICT(simplefunction_module_globals, simplefunction_module_globals_table);
```

Having defined the function object, we have finally to register the function with 

```c
MP_REGISTER_MODULE(MP_QSTR_simplefunction, simplefunction_user_cmodule, MODULE_SIMPLEFUNCTION_ENABLED);
```

This last line is particularly useful, because by setting the `MODULE_SIMPLEFUNCTION_ENABLED` variable in `mpconfigport.h`, you can selectively exclude modules from the compilation, i.e., if in `mpconfigport.h`, which should be in the root directory of the port you want to compile for, 

```c
#define MODULE_SIMPLEFUNCTION_ENABLED (1)
```
then `simplefunction` will be included in the firmware, while with

```c
#define MODULE_SIMPLEFUNCTION_ENABLED (0)
```

the module will be dropped, even though the source is in your modules folder.

## Function names

Whenever you want your function to be visible from the interpreter, you have to add the function object to a dictionary. The dictionary 

```c
STATIC const mp_rom_map_elem_t test_locals_dict_table[] = {
	{ MP_ROM_QSTR(MP_QSTR_somefunction), MP_ROM_PTR(&test_somefunction_obj) },
};


STATIC MP_DEFINE_CONST_DICT(test_locals_dict, test_locals_dict_table);
```

## Parsing arguments

## Parsing keyword arguments

One of the most useful features of python is that functions can accept positional as well as keyword arguments, thereby providing a very flexible tool for supplying the function with value. In this section, we will see how we should set up our function, so that it can interpret what we passed in. 

It does not matter, whether we have positional or keyword arguments, at one point, the interpreter has to turn all arguments into a deterministic sequence of objects. We stipulate this sequence in the constant `mp_arg_t` type variable called `allowed_args[]`. This is nothing but an array of type `mp_arg_t`, which is nothing but two `uint16` values, and a union named `mp_arg_val_t`. This union holds the default value and the type of the variable that we want to pass. (This is all defined in `runtime.h`.) 

After parsing the arguments with `mp_arg_parse_all`, whatever was at the zeroth position of `allowed_args[]` will be called `args[0]`, the object at the first position of `allowed_args[]` will be turned into `args[1]`, and so on. 

```c
STATIC mp_obj_t kw_test(size_t n_args, const mp_obj_t *pos_args, mp_map_t *kw_args) {
    static const mp_arg_t allowed_args[] = {
		{ MP_QSTR_input, MP_ARG_OBJ, {.u_rom_obj = MP_ROM_PTR(&mp_const_none_obj)} }, 
        { MP_QSTR_base,     MP_ARG_INT, {.u_int = 12} },
        { MP_QSTR_mode,     MP_ARG_INT, {.u_int = 555} },
        { MP_QSTR_addr,     MP_ARG_KW_ONLY | MP_ARG_INT, {.u_int = 33} },
        { MP_QSTR_dtype,     MP_ARG_KW_ONLY | MP_ARG_INT, {.u_int = 33} },        
    };

    // parse args
    mp_arg_val_t args[MP_ARRAY_SIZE(allowed_args)];
    mp_arg_parse_all(n_args, pos_args, kw_args, MP_ARRAY_SIZE(allowed_args), allowed_args, args);
    if(MP_OBJ_IS_TYPE(args[0].u_rom_obj, &mp_type_tuple)) {
		printf("tuple!!!");
	}
    printf("base: %lu\n\r",  args[1].u_int);
    printf("mode: %lu\n\r",  args[2].u_int);
    printf("address: %lu\n\r",  args[3].u_int);    
    return mp_const_none;
}

STATIC MP_DEFINE_CONST_FUN_OBJ_KW(kw_test_obj, 1, kw_test);
```

# Working with classes

Of course, python would not be python without classes. A module can include the implementation of a class. The procedure is similar to the implementation of standard functions, except that we have to define a structure that holds at least a string with the name of the class, a pointer to the initialisation and printout functions, and a local dictionary. A typical structure would look like

```c

STATIC const mp_rom_map_elem_t test_locals_dict_table[] = {
	{ MP_ROM_QSTR(MP_QSTR_method1), MP_ROM_PTR(&test_method1_obj) },
	{ MP_ROM_QSTR(MP_QSTR_method2), MP_ROM_PTR(&test_method2_obj) }
}


const mp_obj_type_t test_type = {
	{ &mp_type_type },
	.name = MP_QSTR_test,
	.print = test_print,
	.make_new = test_make_new,
	.locals_dict = (mp_obj_dict_t*)&test_locals_dict,
};
```

The locals dictionary contains all user-facing methods and constants of the class, while the `test_type` structure's `name` member is what 

## Special methods

Python has a number of special methods, which will make a class behave as a native object. So, e.g., if a class implements the `__add__(self, other)` method, then instances of the class can be added with the `+` operator. Here is an example:

In [4]:
class Adder:
    
    def __init__(self, value):
        self.value = value
        
    def __add__(self, other):
        self.value = self.value + other.value
        return self

a = Adder(1)
b = Adder(2)

c = a + b
c.value

3

Note that, while the above example is not particularly useful, it proves the point: upon calling the `+` operator, the values of `a`, and `b` are added. If we had left out the implementation of the `__add__` method, the python interpreter would not have a clue as to what to do with the objects. You can see for yourself what happens, if one is sloppy:

In [5]:
class Adder:
    
    def __init__(self, value):
        self.value = value

a = Adder(1)
b = Adder(2)

c = a + b
c.value

TypeError: unsupported operand type(s) for +: 'Adder' and 'Adder'

Indeed, we do not support the `+` operator.

Now, the problem is that in the C implementation, these special methods have to be treated in a special way. The naive approach would be to add the pointer to the function to the locals dictionary as 

```c
STATIC const mp_rom_map_elem_t test_locals_dict_table[] = {
	{ MP_ROM_QSTR(MP_QSTR___add__), MP_ROM_PTR(&test_add_obj) },
};
```
but that would not work. Well, this is not entirely true: the `+` operator would not work, but one could still call the method explicitly as

```python
a = Adder(1)
b = Adder(2)

a.__add__(b)

```

Before we can actually add the `+` operator to the class, we should note that there are two kinds of special methods (TODO: THERE ARE MORE, THIS HAS TO BE EXPLAINED A BIT), namely the unary and the binary operators. 

In the first group are those, whose sole argument is the class instance itself. Two frequently used cases are the length operator, `len`, and `bool`. So, e.g., if your class implements the `__len__(self)` method, and the method returns an integer, then you can call the `len` function in the console

```python
len(myclass)
```

In the second category of operators are those, which require a left, as well as a right hand side. An example for this was the `__add__` method in our `Adder` class.



## Defining constants

Constants can be added to the locals dictionary as any other object. So, e.g., if we wanted to define the constant MAGIC, we could do that as follows

```c

#define MAGIC 42

STATIC const mp_rom_map_elem_t test_locals_dict_table[] = {
	{ MP_ROM_QSTR(MP_QSTR_MAGIC), MP_ROM_INT(MAGIC) },
};
```
The constant would then be accessible in the interpreter as

```python

import test

test.MAGIC
```

## The complete code

Having seen how components of a class should be implemented, we can bring all parts together. We will define a class that has two methods, one with positional arguments, and one with keyword arguments, has a class-specific constant, and can deal with the `+` and `*` operators. You should be able to drop this code into your user-defined modules directory, and compile it without difficulties.

```c

```

# Dealing with iterables


In python, an iterable is basically an object that you can have in a `for` loop:

```python
for item in my_iterable:
    print(item)
```

Amongs others, lists, tuples, and ranges are iterables. The key is that these object have a special internal method, and iterator, attached to them. This iterator method is responsible for keeping track of the index during the iteration, and serving the objects in the iterable one by one to the `for` loop. When writing our own iterable, we will look under the hood, and see how this all works at the C level. For now, we are going to discuss only, how we can *consume* the content of an iterable in the C code. 


In order to demonstrate the use of an iterator, we are going to write a function that takes the values from a list, and squares them. 





If you want to find out, whether a particular variable is an iterable, you can 


`objlist.c`, `objtuple.c`, `objarray.c`, `objrange.c` etc.

# Profiling

There are times, when you might want to find out how long a particular operation takes. If you are interested in the execution time of a complete function, you can measure it simply by making use of the python interpreter

```python

from utime import ticks_us, diff_ticks

now = ticks_us
run_my_function()
then = diff_ticks(ticks_us(), now)

print("function run_my_function() took %d us to run"%then)

```

In fact, since our function is flanked by two other statements, this construct easily lends itself to a decorator implementation. 

(If you need an even better estimate, you can yank run_my_function(): in this way, you would get the costs of measuring time itself:

```python

from utime import ticks_us, diff_ticks

now = ticks_us
then = diff_ticks(ticks_us(), now)

print("the time measurement took %d us"%then)

```
Then you subtract the results of the second measurement from those of the first.)